<font size="-1">
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
            <b>3D Object Representations for Fine-Grained Categorization</b><br>
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
            Jonathan Krause, Michael Stark, Jia Deng, Li Fei-Fei<br>
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
            <i>4th IEEE Workshop on 3D Representation and Recognition, at ICCV 2013</i> <b>(3dRR-13)</b>. Sydney, Australia. Dec. 8, 2013.<br>
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
            <a style="text-decoration: underline" href="../papers/3drr13.pdf">[pdf]</a>
            &nbsp;&nbsp;<a style="text-decoration: underline" href="../papers/3drr13.bib">[BibTex]</a>
            &nbsp;&nbsp;<a style="text-decoration: underline" href="../papers/3drr_talk.pdf">[slides]</a>
</font>

Pour créer un env dédié:
conda create -n cars numpy=1.19.1 scipy tensorflow keras matplotlib notebook pandas
  
Ressources utilisées :  
https://medium.com/swlh/recognising-cats-and-dogs-using-neural-networks-with-tensorflow-6f366ad30dbf  
https://towardsdatascience.com/how-to-load-matlab-mat-files-in-python-1f200e1287b5  
  
    
Besoin de 3 datasets:
- training 
- validation
- test

Tout est fourni dans le dataset  choisi

dataset from https://ai.stanford.edu/~jkrause/cars/car_dataset.html  
NB: Does not feature a lot of french car brands

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os

print("imports ok")

imports ok


In [37]:
#import des infos sur les classes de voitures
from scipy.io import loadmat
meta_annot = loadmat('car_devkit\cars_meta.mat')
data = [[row.flat[0] for row in line] for line in meta_annot['class_names'][0]]
print(len(data))
columns = ["model_id", "class_name",]
df_names = pd.DataFrame(list(zip(range( 1,len(data)+1), data)), columns=columns)
df_names.head()

196


,model_id,class_name
0,1,[AM General Hummer SUV 2000]
1,2,[Acura RL Sedan 2012]
2,3,[Acura TL Sedan 2012]
3,4,[Acura TL Type-S 2008]
4,5,[Acura TSX Sedan 2012]


In [39]:
#Il faut maintenant regrouper les classes de véhicules par marque au lieu du modèle
df_names["cnames"] = df_names.class_name.astype(str).apply(lambda x: x.split(" ")[0].replace("['",""))
df_names["cnames"] = df_names["cnames"].apply(lambda x: "AM General" if x=="AM" else x)
df_names["brand_id"] = pd.factorize(df_names["cnames"])[0]
set(df_names.cnames.tolist()) #visualisation de la liste des marques
df_names
#créer dictionnaire pour convertir un type d'identifiant en un autre ?
# (lambda x: x.split(" ").lower()[0] for x in line)

ValueError: Length of values (2) does not match length of index (196)

In [43]:
#import des annotations de training depuis le fichier matlab
from scipy.io import loadmat

mat_annot = loadmat('car_devkit\cars_train_annos.mat')
data = [[row.flat[0] for row in line] for line in mat_annot['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'fname']
df_train = pd.DataFrame(data, columns=columns)
df_train.head()

,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,fname
0,39,116,569,375,14,00001.jpg
1,36,116,868,587,3,00002.jpg
2,85,109,601,381,91,00003.jpg
3,621,393,1484,1096,134,00004.jpg
4,14,36,133,99,106,00005.jpg


In [47]:
#vérifier que les ids des classes correspondent bien entre eux
if (df_train["class"].min() != df_names["class_id"].min()):
    print("Err au niveau des min")
elif (df_train["class"].max() != df_names["class_id"].max()):
    print("Err au niveau des max")
else:
    print("Identifiants ok")
    

Identifiants ok


In [52]:
#import des infos de test 
from scipy.io import loadmat
test_annot = loadmat('car_devkit\cars_test_annos.mat')
data = [[row.flat[0] for row in line] for line in test_annot['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'fname']
df_test = pd.DataFrame(data, columns=columns)
df_test.head()
#we are not sending our results -> this df is useless

,bbox_x1,bbox_y1,bbox_x2,bbox_y2,fname
0,30,52,246,147,00001.jpg
1,100,19,576,203,00002.jpg
2,51,105,968,659,00003.jpg
3,67,84,581,407,00004.jpg
4,140,151,593,339,00005.jpg
